In [12]:
import sys
sys.path.append('/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/src/RAG')
from vector_store_utils import *
import sys
sys.path.append('/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/src')
from utils import *

In [13]:
alll = getData("/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/dataset/课本md/高中/md/合并/2.加课外内容.json")
alll[0]

{'id': 'qvrr9a7mfp059p452bfs',
 'title': '# 独立权 管辖权 平等权 自卫权',
 'content': '\n◆ 独立权，是指主权国家拥有按照自己的意志处理内政、外交事务而不受他国控制和干涉的权利。  \n\n◆ 管辖权，是指主权国家对其领域内的一切人和物具有管辖的权利。  \n\n◆ 平等权，是指主权国家不论大小、强弱，也不论政治、经济、意识形态和社会制度有何差异，在国际法上的地位一律平等。  \n\n◆ 自卫权，是指主权国家拥有保卫自己的生存和独立的权利。  \n',
 'grade': '选择性必修1',
 'subject': '高中政治',
 'modify_content': '独立权，是指主权国家拥有按照自己的意志处理内政、外交事务而不受他国控制和干涉的权利。\n\n管辖权，是指主权国家对其领域内的一切人和物具有管辖的权利。\n\n平等权，是指主权国家不论大小、强弱，也不论政治、经济、意识形态和社会制度有何差异，在国际法上的地位一律平等。\n\n自卫权，是指主权国家拥有保卫自己的生存和独立的权利。',
 '课本内容': '独立权，是指主权国家拥有按照自己的意志处理内政、外交事务而不受他国控制和干涉的权利。\n\n管辖权，是指主权国家对其领域内的一切人和物具有管辖的权利。\n\n平等权，是指主权国家不论大小、强弱，也不论政治、经济、意识形态和社会制度有何差异，在国际法上的地位一律平等。\n\n自卫权，是指主权国家拥有保卫自己的生存和独立的权利。',
 '课外内容': '在国际法框架下，主权国家四大基本权利的现实实践具有多维度的体现。以独立权为例，2023年不结盟运动峰会上，印度尼西亚公开拒绝大国干预其镍矿出口政策，援引《联合国宪章》第二条第七款阐明其经济自主权。管辖权在孟晚舟引渡案中得到具象化呈现：加拿大依据属地管辖原则实施司法程序，而中国则启动领事保护机制主张属人管辖权，该案件成为研究国家管辖权竞合冲突的经典案例。平等权的当代实践体现于2022年联合国大会表决俄国决议草案时，圣多美和普林西比（人口21万）与印度（13亿人口）享有同等表决权重，这一制度设计深刻反映了主权平等原则。自卫权的法律边界在2020年伊朗圣城旅指挥官苏莱曼尼遇袭事件中引发全球讨论，多国法学家援引《联合国宪章》第51条，就预防性自卫的合法性

In [14]:
len(alll)

4107

In [15]:
data_type = "从全部数据导入"
import os
if data_type == "从未清洗数据得到":
    subjects = ["高中生物","高中历史","高中地理","高中政治"]
    all_res  = []
    all_text = []
    for sub in subjects:
        paths = get_absolute_file_paths(os.path.join("/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/dataset/课本md/初中",sub),"json")
        for path in paths:
            grade = path.split("/")[-1].split(" ")[-1].split(".")[0]
            res = getData(path)
            new_res = []
            new_text = []
            for item in res:
                new_res.append({"subject":sub,"grade":grade,"title":item['title'].strip(),"content":item['content'].strip()})
                new_text.append(item['content'].strip())
            all_res.extend(new_res)
            all_text.extend(new_text)
else:
    alll = getData("/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/dataset/课本md/高中/md/合并/2.加课外内容.json")
    all_res  = []
    all_text = []
    for item in alll:
        all_res.append({"subject":item['subject'].strip(),"grade":item['grade'].strip(),"title":item['title'].strip(),"content":item['课本内容'].strip()})
        all_text.append(item['content'].strip())



In [16]:
from milvus_model.hybrid import BGEM3EmbeddingFunction

ef = BGEM3EmbeddingFunction(
    model_name = "/hpc2hdd/home/fye374/models/BAAI/bge-m3",
    use_fp16=False, 
    device="cuda:0")
dense_dim = ef.dim["dense"]

docs_embeddings = ef(all_text)


pre tokenize: 100%|██████████| 257/257 [00:00<00:00, 459.45it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 257/257 [00:35<00:00,  7.30it/s]


In [17]:
import sys
sys.path.append("/hpc2hdd/home/fye374/ZWZ_Other/quizmanus")
from src.config.rag import DB_URI,COLLECTION_NAME
from src.RAG.vector_store_utils import *
db_uri = DB_URI
col_name = COLLECTION_NAME
col = get_collection(db_uri,col_name)

In [ ]:
# collection_info = col.describe()
# # collection_info

In [ ]:
# iterator = col.query_iterator(
#     batch_size=10,
#     expr=None,
#     output_fields=["subject"]
# )

# value_set = set()

# while True:
#     result = iterator.next()
#     if not result:
#         iterator.close()
#         break
    
#     for item in result:
#         value_set.add(item["subject"])
# col.release()

In [ ]:
# value_set

In [18]:
col = create_collection(db_uri,col_name, dense_dim)

collection subjects already exists, 直接返回已有subjects


In [ ]:
all_res[0]

In [19]:
add_data(col, all_res,docs_embeddings)

100%|██████████| 4107/4107 [00:49<00:00, 83.30it/s] 

添加成功


# 检索示例

In [ ]:
print(

In [20]:
from milvus_model.hybrid import BGEM3EmbeddingFunction

ef = BGEM3EmbeddingFunction(
    model_name = "/hpc2hdd/home/fye374/models/BAAI/bge-m3",
    use_fp16=False, 
    device="cuda:0")

In [21]:
# query = input("Enter your search query: ")
# print(query)
query = "生物微分子"
query_embeddings = ef([query])


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [22]:
import sys
sys.path.append('/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/src/RAG')
from retrieval import *
from reranker import *

In [23]:
db_uri = "/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/src/RAG/vector_store/subjects.db"
col_name = "subjects"
col = get_collection(db_uri,col_name)
dense_results = dense_search(col, query_embeddings["dense"][0])
sparse_results = sparse_search(col, query_embeddings["sparse"]._getrow(0))
hybrid_results = hybrid_search(
    col,
    query_embeddings["dense"][0],
    query_embeddings["sparse"]._getrow(0),
    sparse_weight=0.7,
    dense_weight=1.0,
)
# /hpc2hdd/home/fye374/ZWZ_Other/quizmanus/src/RAG/vector_store

In [24]:
rerank(query, hybrid_results)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


['微生物是难以用肉眼观察的微小生物的统称，包括细菌、真菌、病毒及一些原生生物等。本章中提及的微生物主要指用于发酵的细菌和真菌。',
 '微生物是难以用肉眼观察的微小生物的统称，包括细菌、真菌、病毒及一些原生生物等。本章中提及的微生物主要指用于发酵的细菌和真菌。  \n\n图1-6 盛有液体培养基的锥形瓶（左）和长有酵母菌菌落的固体培养基（右）  \n\n为什么培养基需要氮源？',
 '科学家访谈 探究微观生命世界的奥秘与施一公院士一席谈第 1 章 走近细胞.第 1 节 细胞是生命活动的基本单位..第2 节 细胞的多样性和统一性..探究・实践 使用高倍显微镜观察几种细胞.生物科技进展 人工合成生命的探索. 12  \n\n\n第 2 章 组成细胞的分子. 15  \n第1 节 细胞中的元素和化合物.... ..16  \n探究・实践 检测生物组织中的糖类、脂肪和蛋白质.........18  \n第2 节 细胞中的无机物..... ..20  \n第 3 节 细胞中的糖类和脂质..... ..23  \n第4 节 蛋白质是生命活动的主要承担者... ..28  \n生物科学史话 世界上第一个人工合成蛋白质的诞生.........33  \n第5 节 核酸是遗传信息的携带者...... ..34  \n\n\n\n第 3 章 细胞的基本结构. （第 1 节 细胞膜的结构和功能. 40第 2 节 细胞器之间的分工合作..  \n\n探究・实践 用高倍显微镜观察叶绿体和细胞质的流动.....50  \n第3 节 细胞核的结构和功能. 54  \n探究・实践 尝试制作真核细胞的三维结构模型 57  \n生物科技进展 世界上首例体细胞克隆猴的诞生. 58  \n第 4 章 细胞的物质输入和输出 61  \n第1 节 被动运输. 62  \n探究・实践 探究植物细胞的吸水和失水.... 64  \n生物科学史话 人类对通道蛋白的探索历程.... .68  \n第2 节 主动运输与胞吞、胞吐.... .69']

In [25]:
hybrid_results

['微生物是难以用肉眼观察的微小生物的统称，包括细菌、真菌、病毒及一些原生生物等。本章中提及的微生物主要指用于发酵的细菌和真菌。  \n\n图1-6 盛有液体培养基的锥形瓶（左）和长有酵母菌菌落的固体培养基（右）  \n\n为什么培养基需要氮源？',
 '微生物是难以用肉眼观察的微小生物的统称，包括细菌、真菌、病毒及一些原生生物等。本章中提及的微生物主要指用于发酵的细菌和真菌。',
 '蛋白质的基本单位是氨基酸，组成核酸的基本单位是核苷酸，这些基本单位称为单体。每一个单体都以若干个相连的碳原子构成的碳链为基本骨架。生物大分子是由许多单体连接成的多聚体（图2-17），因此，生物大分子也是以碳链为基本骨架的。正是由于碳原子在组成生物大分子中的重要作用，科学家才说“碳是生命的核心元素”“没有碳，就没有生命”。  \n\n图2-17   生物大分子是由许多单体连接成的多聚体  \n\n以碳链为骨架的多糖、蛋白质、核酸等生物大分子，构成细胞生命大厦的基本框架；糖类和脂质提供了生命活动的重要能源；水和无机盐与其他物质一起，共同承担着构建细胞、参与细胞生命活动等重要功能。细胞中的这些化合物，含量和比例处在不断变化之中，但又保持相对稳定，以保证细胞生命活动的正常进行。',
 '蛋白质的基本单位是氨基酸，组成核酸的基本单位是核苷酸，这些基本单位称为单体。每一个单体都以若干个相连的碳原子构成的碳链为基本骨架。生物大分子是由许多单体连接成的多聚体，因此，生物大分子也是以碳链为基本骨架的。正是由于碳原子在组成生物大分子中的重要作用，科学家才说“碳是生命的核心元素”“没有碳，就没有生命”。\n\n以碳链为骨架的多糖、蛋白质、核酸等生物大分子，构成细胞生命大厦的基本框架；糖类和脂质提供了生命活动的重要能源；水和无机盐与其他物质一起，共同承担着构建细胞、参与细胞生命活动等重要功能。细胞中的这些化合物，含量和比例处在不断变化之中，但又保持相对稳定，以保证细胞生命活动的正常进行。',
 '在自然界中，微生物的种类远远超过动植物的种类。科学家预测，目前在实验室里能够培养的微生物的种类还不到自然界中存在的微生物种类的1%，甚至更低。在如此庞大的微生物王国里，研究者如何获得目标菌种呢？  \n\n过去，人们通过手工操作来进行筛选，不仅费时费力，而且大大限制了筛选量。微  \n\n生物菌种的高通量筛选以有多